In [ ]:
import pandas as pd
import altair as alt
import vega_datasets
alt.data_transformers.disable_max_rows()

In [7]:
hatecrimes = pd.read_csv("../data/hate_crime.csv")

In [ ]:
alt.Chart(hatecrimes).mark_rect().encode(
    x='date(incident_date):O',
    y='month(incident_date):O',
    color='count()'
).display()

In [ ]:
alt.Chart(hatecrimes).mark_line().encode(
        x='data_year:O',
        y = 'count():N'
    ).display()

In [ ]:
alt.Chart(hatecrimes).mark_line().encode(
        x='data_year:O',
        y = 'count():N',
        color='bias_desc:N'
    ).display()

In [ ]:
# Cleaning pandas 'bias description' variable
#hatecrimes.groupby(['bias_desc']).size().sort_values(ascending=False).head(10)
# Most popular hate crimes are anti Black, Jewish, White, Gay, Hispanic, Other Race, LGBT, Asian, Multi-Race, Lesbian

In [ ]:
top10 = ["Anti-Black or African American", "Anti-Jewish", "Anti-White", "Anti-Gay (Male)", "Anti-Hispanic or Latino", "Anti-Other Race/Ethnicity/Ancestry","Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)", "Anti-Asian", "Anti-Multiple Races, Group", "Anti-Lesbian (Female)"]
hatecrimestop10 = hatecrimes.loc[hatecrimes['bias_desc'].isin(top10)]
alt.Chart(hatecrimestop10).mark_line().encode(
        x='data_year:O',
        y = 'count():N',
        color='bias_desc:N'
    ).display()

In [ ]:
# Here we can see that Anti-Black hate crimes are far more prevalent than other hate crimes, but have decreased from 2022-2023. Anti-Jewish hate crimes on the other hand, have risen since 2021

In [4]:
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature='states')

In [5]:
pop = data.population_engineers_hurricanes(usecols=['state', 'id'])

In [6]:
hatecrimes_w_ids = hatecrimes.merge(pop, left_on='state_name', right_on='state')

In [ ]:
states = alt.topo_feature(data.us_10m.url, 'states')
alt.Chart(states).mark_geoshape().encode(
    color='size:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_counts, 'id', list(state_counts.columns))
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

In [ ]:
# Total heatmap of all hatecrimes in continental US -- consider adjusting for population

In [ ]:
hatecrimes.groupby(['offense_name']).size().sort_values(ascending=False).head(10)

In [ ]:
alt.Chart(hatecrimes).mark_bar().encode(
        x='offender_race:O',
        y = 'count():N'
    ).display()

In [9]:
# Tracking hate crimes specifically in 2019-2023
years = [2019, 2020, 2021, 2022, 2023]
hatecrimes1924= hatecrimes.loc[hatecrimes['data_year'].isin(years)]

In [10]:
hatecrimes1924.groupby(['bias_desc']).size().sort_values(ascending=False).head(10)
top10_rec = ["Anti-Black or African American", "Anti-Jewish", "Anti-White", "Anti-Gay (Male)", "Anti-Hispanic or Latino", "Anti-Other Race/Ethnicity/Ancestry","Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)", "Anti-Asian", "Anti-Multiple Races, Group", "Anti-Transgender"]
hatecrimestop10_rec = hatecrimes1924.loc[hatecrimes1924['bias_desc'].isin(top10_rec)]


In [ ]:
main_c = alt.Chart(hatecrimestop10_rec).mark_line().encode(
        x='yearmonth(incident_date):O',
        y = 'count()', 
        color='bias_desc:N'
    )

text1 = alt.Chart({'values': [{'x': 100, 'y': 500}]}).mark_text(
    text="Murder of George Floyd").encode(
        alt.X('x:Q', axis=None),
        y='y:Q')

text2 = alt.Chart({'values': [{'x': 260, 'y': 150}]}).mark_text(
    text="2021 Atlanta Spa Shootings").encode(
        alt.X('x:Q', axis=None),
        y='y:Q')

text3 = alt.Chart({'values': [{'x': 640, 'y': 300}]}).mark_text(
    text="Hamas Attacks Israel").encode(
        alt.X('x:Q', axis=None).scale(domain=(-100, 100)),
        y='y:Q')

text = text3 + text2 + text1
main_c + text
# Want to add text here to note particular events that saw a spike -- George Floyd's murder, the 2021 Atlanta spa shootings, Oct. 7